In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from preprocess import listFiles
from register import loadData


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# file_path = 'datasets/json/box_plant1/box_plant1_frame300_400.json'
# # Open and read the JSON file
# with open(file_path, 'r') as file:
#     data = json.load(file)

# # Now 'data' is a dictionary
# num_key = len(data.keys())
# if num_key > 0:
#     print(f"Data loaded, number of frames = {num_key}")
# frame = 350
# points = np.array(data[str(frame)])

Data loaded, number of frames = 100


In [2]:
def quickVisualize(points):
    """ Visualize pointcloud from numpy array"""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd])
    return

In [3]:
### for handling entier point cloud
def shortPassFilter(points, threshold = 10):
    """ Filter points that are further than a distance away"""
    distance = (points[:,0]**2 + points[:,1]**2 + points[:,2]**2)**0.5
    mask = distance < threshold
    return points[mask]

# prepare path
directory = r'datasets/json/box_plant1/'
files = listFiles(directory)
files.sort()

# prepare frame 0 as reference
path = directory + files[0]
with open(path, 'r') as file:
    data = json.load(file)
pts_frame_0 = np.array(data[str(0)])
pts_frame_60 = np.array(data[str(60)])
pts_frame_0 = shortPassFilter(pts_frame_0, 7)
pts_frame_60 = shortPassFilter(pts_frame_60, 7)


In [27]:
### Global Registration
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_plotly([source_temp, target_temp])


def preprocess_point_cloud(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh


def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result



In [40]:
path1 = r'datasets/csv/box_plant1_manual/box_plant1_manual_frame5.csv'
path2 = r'datasets/csv/box_plant1_manual/box_plant1_manual_frame10.csv'
source = loadData(path1)
target = loadData(path2)

voxel_size=0.05
source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)

print("Apply fpfh registration")
result_ransac = execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size)

### Applying ICP for local registration
trans_init = np.asarray(result_ransac.transformation)
threshold=0.02
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)


# draw_registration_result(source_down, target_down, result_ransac.transformation)

Apply fpfh registration
Apply point-to-point ICP
RegistrationResult with fitness=4.342273e-01, inlier_rmse=1.367689e-02, and correspondence_set size of 1360
Access transformation to get result.
Transformation is:
[[ 0.99639127 -0.06612389 -0.05321709  0.06309249]
 [ 0.06193094  0.9951123  -0.07691603  0.14414297]
 [ 0.05804297  0.07334267  0.99561632 -0.02037872]
 [ 0.          0.          0.          1.        ]]


In [8]:
from register import fullRegistration, loadData

In [9]:
### Choose directory here
directory = r'datasets/csv/box_plant1_manual/'
files = listFiles(directory)
files.sort()
pcds = []
for file in files:
    path = directory + file
    pcds.append(loadData(path))

pose_graph = fullRegistration(pcds)
breakpoint()

Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.pipelines.reg

In [24]:
# Apply the transformations to each point cloud
aligned_pcds = []
for i, node in enumerate(pose_graph.nodes):
    # Get the transformation matrix
    transformation = node.pose
    pcd_temp = copy.deepcopy(pcds[i])
    pcd_temp.transform(transformation)
    # pcd_temp.transform(np.eye(4))
    temp = [2,3,4,5,6,7,8,9,10,11, 15, 16]
    if True:
        aligned_pcds.append(pcd_temp)

combined_pcd = o3d.geometry.PointCloud()
for pcd in aligned_pcds:
    combined_pcd += pcd

combined_pcd_downsampled = combined_pcd.voxel_down_sample(voxel_size=0.02)
o3d.visualization.draw_geometries([combined_pcd_downsampled])

# Save the combined point cloud
# o3d.io.write_point_cloud("combined_point_cloud.ply", combined_pcd_downsampled)